In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data= pd.read_excel("/kaggle/input/yamana-gold-inc-stock-price/Yamana_Gold_Inc._AUY.csv.xlsx")

data.head()

In [ ]:
data.tail()

In [ ]:
# Ensure the data is sorted by date
data = data.sort_values(by='Date')

# Calculate the 50-day and 200-day simple moving averages
data['SMA50'] = data['Close'].rolling(window=50).mean()
data['SMA200'] = data['Close'].rolling(window=200).mean()

# Calculate the 50-day and 200-day exponential moving averages
data['EMA50'] = data['Close'].ewm(span=50, adjust=False).mean()
data['EMA200'] = data['Close'].ewm(span=200, adjust=False).mean()


In [ ]:
import numpy as np

def backtest_moving_average(data):
    data['Signal'] = 0
    data['Signal'][50:] = np.where(data['SMA50'][50:] > data['SMA200'][50:], 1, -1)
    data['Position'] = data['Signal'].shift()

    # Calculate returns
    data['Returns'] = data['Close'].pct_change()
    data['Strategy Returns'] = data['Returns'] * data['Position']

    # Calculate cumulative returns
    data['Cumulative Returns'] = (1 + data['Strategy Returns']).cumprod()
    return data

# Backtest the moving average strategy
data = backtest_moving_average(data)


In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

# Ensure data is in chronological order
data = data.sort_values(by='Date')

# Candlestick chart with moving averages
fig = go.Figure(data=[go.Candlestick(x=data['Date'],
                                     open=data['Open'],
                                     high=data['High'],
                                     low=data['Low'],
                                     close=data['Close']),
                      go.Scatter(x=data['Date'], y=data['SMA50'], mode='lines', name='SMA50'),
                      go.Scatter(x=data['Date'], y=data['SMA200'], mode='lines', name='SMA200')])

fig.update_layout(title='Yamana Gold Inc. Stock Price with Moving Averages',
                  xaxis_title='Date',
                  yaxis_title='Price (USD)',
                  width=1400, height=700)  # Adjust dimensions for larger display
fig.show()

# Plot cumulative returns
plt.figure(figsize=(16, 8))  # Adjust dimensions for larger display
plt.plot(data['Date'], data['Cumulative Returns'], label='Strategy Returns')
plt.title('Cumulative Returns of Moving Average Strategy')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.show()
